In [4]:
#ENV["JULIA_DEBUG"] = Main

In [5]:
#ENV["JULIA_DEBUG"] = "none"

In [6]:
using AppleAccelerate

In [7]:
using LinearAlgebra, Polyhedra, CDDLib
using JuMP, Ipopt, Clarabel
using Random
using PhyloNetworks

In [8]:
function trop_normalize(x)
    return x .- first(x)
end

trop_normalize (generic function with 1 method)

# Polyhedral point location

In [9]:
"""
Calculate the polyhedral distance between two vectors.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
"""
function poly_dist(vec1, vec2, alphas)
    differences = alphas * (vec1 - vec2)
    return maximum(differences)
end

poly_dist

In [10]:
"""
Calculate the sum of polyhedral distances between `ref` and the points in `sample`.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function sum_of_poly_dist(ref, sample, alphas; power=1)
    return sum([poly_dist(pt, ref, alphas)^power for pt in sample])
end

sum_of_poly_dist

In [11]:
function poly_frechet_model(sample, alphas; power=2)
    dim = length(sample[1])
    
    # Choose model depending on power
    if power == 1
        error("FW computation not yet implemeneted")
    elseif power == 2
        model = Model(Clarabel.Optimizer)
    else
        model = Model(Ipopt.Optimizer)
    end
    
    # suppress printing
    set_silent(model)

    @variable(model, x[1:dim])
    @variable(model, t[1:length(sample)])

    @objective(model, Min, sum(t))

    for (p_idx, p) in enumerate(sample)
        expressions = alphas * (x - p)
        
        for expr in expressions
            @constraint(model, t[p_idx] >= (expr)^power)
        end
    end

    return model, x
end

poly_frechet_model (generic function with 1 method)

In [12]:
"""
Find one polyhedral Fréchet mean of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function poly_frechet(sample, alphas; power=2)
    model, x = poly_frechet_model(sample, alphas, power=power)
    
    @debug "\nOptimising..."
    
    optimize!(model)
    minimiser = value.(x)
    
    return minimiser
end

poly_frechet

In [13]:
"""
Find the set of polyhedral Fréchet means of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
`rep` is either "vrep" or "hrep" -- returning either vertices or halfspaces.
"""
function poly_frechet_set(::Type{T}, sample, alphas; power=2, rep=:polyhedron, tol=1e-3) where {T<:Polyhedra.Library}
    dim = length(sample[1])
    num_facets = size(alphas)[1]
    
    # Compute one Fréchet mean
    one_mean = poly_frechet(sample, alphas, power=power)
    
    @debug "Frechet mean found: $(one_mean)"
    
    # Rationalise all coordinates
    rat_alphas = rationalize.(alphas, tol=tol)
    rat_sample = [rationalize.(pt, tol=tol) for pt in sample]
    rat_mean = rationalize.(one_mean, tol=tol)
    
    distances = [poly_dist(rat_mean, pt, alphas) for pt in rat_sample]
    
    Amat = vcat(rat_alphas, -rat_alphas)
    bval1 = Rational{Int64}[]
    bval2 = Rational{Int64}[]
    
    # Discard redundant halfspacesprintln("Setting up constraints: $(round(progress/total * 100, digits=3))%   \r")
    evals = rat_alphas * hcat([rat_mean - pt for pt in rat_sample]...)
    for k in 1:length(rat_sample)
        evals[:,k] .-= distances[k]
    end
    
    progress = 0
    total = num_facets
    
    for k = 1:num_facets
        
        greatest_nonpos = argmax([x > 0 ? -Inf : x for x in evals[k,:]])
        push!(bval1, dot(rat_alphas[k,:], rat_sample[greatest_nonpos]) + 
            distances[greatest_nonpos])
        
        progress += 1
        @debug "Removing redundant half-spaces: $(round(progress/total * 100, digits=3))%   \r"    
    end
    
    @debug "\nFinding defining facets..."
    
    poly = polyhedron(hrep(rat_alphas, bval1), T(:exact))
    removehredundancy!(poly)
    
    if rep == :hrep
        @debug "Finding facets..."
        return hrep(poly)
    elseif rep == :vrep
        @debug "Finding vertices..."
        return vrep(poly)
    else
        @debug "Defaulting to polyhedron."
        return poly
    end
end

poly_frechet_set(sample, alphas; power=2, rep=:polyhedron, tol=1e-3) = poly_frechet_set(CDDLib.Library, sample, alphas, power=power, rep=rep, tol=tol)

poly_frechet_set (generic function with 2 methods)

## Examples

### Toy example: Fréchet mean for $L^\infty$-norm in $\mathbb{R}^2$

In [14]:
### Facets for the L-infinity norm
### Technically twice as long as strictly necessary
Linf_facets = [ 1  0 ;
               -1  0 ;
                0  1 ;
                0 -1 ]

sample = [ [0, 0], 
           [2, 2] ]

my_point = [7, 10]

println("Polyhedral distances from sample to $(my_point): ", 
    [poly_dist(pt, my_point, Linf_facets) for pt in sample])

println("Sum of polyhedral distances: ", 
    sum_of_poly_dist(my_point, sample, Linf_facets))

println("Sum of squared polyhedral distances: ", 
    sum_of_poly_dist(my_point, sample, Linf_facets, power=2))


Polyhedral distances from sample to [7, 10]: [10, 8]
Sum of polyhedral distances: 18
Sum of squared polyhedral distances: 164


In [15]:
@show poly_frechet(sample, Linf_facets)
poly_frechet_set(sample, Linf_facets, rep="vrep", tol=1e-3)

poly_frechet(sample, Linf_facets) = [1.000052213466546, 1.000052213466426]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
2-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, 0], 1//1)
 HyperPlane(Rational{BigInt}[0, 1], 1//1)

### Manual tropical Fréchet mean in $\mathbb{R}^3/\mathbb{R}\mathbf{1}$

In [16]:
Linf3_facets = [ -1   1  0;
                  1  -1  0;
                 -1   0  1;
                  1   0 -1;
                  0  -1  1;
                  0   1 -1 ]

sample2 = [ [0, 0, 0], 
            [0, 2, 4],
            [0, 5, 1] ]

@show poly_frechet(sample2, Linf3_facets, power=2) |> trop_normalize
poly_frechet_set(sample2, Linf3_facets, rep="vrep", tol=1e-3)

poly_frechet(sample2, Linf3_facets, power = 2) |> trop_normalize = [0.0, 1.9999999991413533, 1.0001319108600755]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
2-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[-1, 1, 0], 2//1)
 HyperPlane(Rational{BigInt}[1, 0, -1], -1//1)

# Tropical Frèchet means

In [17]:
"""
Find the relevant facet normals in n dimensions for a tropical ball
"""
function trop_facets(n::Int64)
    result = zeros(Rational{Int64}, n * (n - 1), n)
    k = 1
    for i = 1:n
        for j = 1:n
            if i != j
                result[k, i] = 1//1
                result[k, j] = -1//1
                k += 1
            end
        end
    end
    return result
end

trop_facets

In [18]:
"""
Calculate the tropical distance between two vectors.
"""
function trop_dist(vec1, vec2)
    return maximum(vec1 - vec2) - minimum(vec1 - vec2)
end

trop_dist

In [19]:
"""
Calculate the sum of tropical distances between `ref` and the points in `sample`.
"""
function sum_of_trop_dist(ref, sample; power=1)
    return sum([trop_dist(pt, ref)^power for pt in sample])
end

sum_of_trop_dist

In [20]:
"""
Find one polyhedral Fréchet mean of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function trop_frechet(sample; power=2)
    dim = length(sample[1])
    alphas = trop_facets(dim)
    return poly_frechet(sample, alphas, power=power)
end

trop_frechet

In [21]:
"""
Find one polyhedral Fréchet mean of a given sample.
The rows of `alphas` are the facet normals scaled to α⋅x = 1.
`power` gives the exponent of the distance before taking the sum.
"""
function trop_frechet_set(sample; power=2, rep=:polyhedron, tol=1e-3)
    dim = length(sample[1])
    alphas = trop_facets(dim)
    return poly_frechet_set(sample, alphas, power=power, rep=rep, tol=tol)
end

trop_frechet_set

## Examples
### Toy example

In [22]:
sample = [[0,0,0], [0,4,1]]

my_point = [7, 10, 2]

println("Tropical distances: ", 
    [trop_dist(pt, my_point) for pt in sample])

println("Sum of tropical distances: ", 
    sum_of_trop_dist(my_point, sample))

println("Sum of squared tropical distances: ", 
    sum_of_trop_dist(my_point, sample, power=2))

Tropical distances: [8, 6]
Sum of tropical distances: 14
Sum of squared tropical distances: 100


In [23]:
num_FM = trop_frechet(sample) |> trop_normalize
@show num_FM
trop_frechet_set(sample, rep="vrep", tol=1e-3)

num_FM = [0.0, 2.0000154234464373, 0.7004901999320469]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
1-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, -1, 0], -2//1),
2-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[0, 1, -1], 2//1)
 HalfSpace(Rational{BigInt}[0, -1, 1], -1//1)

### Example of Section 4.2. "Exact Quadratic Optimization"

In [24]:
sample = [[-3,0,0], [0,-6,0], [0,0,-12]]

num_FM = trop_frechet(sample) |> trop_normalize
@show num_FM
trop_frechet_set(sample, rep="vrep", tol=1e-3)

num_FM = [0.0, -1.4387319974273893e-8, -1.0001294161599674]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
2-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, 0, -1], 1//1)
 HyperPlane(Rational{BigInt}[0, 1, -1], 1//1)

In [25]:
trop_facets(3)

6×3 Matrix{Rational{Int64}}:
  1  -1   0
  1   0  -1
 -1   1   0
  0   1  -1
 -1   0   1
  0  -1   1

### Example 10 (Failure of Sturm's algorithm)

In [26]:
sample = [ [0, 0, 0], 
           [0, 2, 4],
           [0, 5, 1] ]

num_FM = trop_frechet(sample) |> trop_normalize
@show num_FM
trop_frechet_set(sample, rep="vrep", tol=1e-3)

num_FM = [0.0, 1.9999999991422408, 1.0001320211409572]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
2-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, -1, 0], -2//1)
 HyperPlane(Rational{BigInt}[1, 0, -1], -1//1)

In [27]:
model = poly_frechet_model(sample, trop_facets(3), power=2) |> first
display(model)
constraints_string(MIME("text/plain"), model)

A JuMP Model
├ solver: Clarabel
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 6
├ num_constraints: 18
│ └ QuadExpr in MOI.GreaterThan{Float64}: 18
└ Names registered in the model
  └ :t, :x

18-element Vector{String}:
 "-x[1]² + 2 x[2]*x[1] - x[2]² + t[1] ≥ 0"
 "-x[1]² + 2 x[3]*x[1] - x[3]² + t[1] ≥ 0"
 "-x[1]² + 2 x[2]*x[1] - x[2]² + t[1] ≥ 0"
 "-x[2]² + 2 x[3]*x[2] - x[3]² + t[1] ≥ 0"
 "-x[1]² + 2 x[3]*x[1] - x[3]² + t[1] ≥ 0"
 "-x[2]² + 2 x[3]*x[2] - x[3]² + t[1] ≥ 0"
 "-x[1]² + 2 x[2]*x[1] - x[2]² - 4 x[1] + 4 x[2] + t[2] ≥ 4"
 "-x[1]² + 2 x[3]*x[1] - x[3]² - 8 x[1] + 8 x[3] + t[2] ≥ 16"
 "-x[1]² + 2 x[2]*x[1] - x[2]² - 4 x[1] + 4 x[2] + t[2] ≥ 4"
 "-x[2]² + 2 x[3]*x[2] - x[3]² - 4 x[2] + 4 x[3] + t[2] ≥ 4"
 "-x[1]² + 2 x[3]*x[1] - x[3]² - 8 x[1] + 8 x[3] + t[2] ≥ 16"
 "-x[2]² + 2 x[3]*x[2] - x[3]² - 4 x[2] + 4 x[3] + t[2] ≥ 4"
 "-x[1]² + 2 x[2]*x[1] - x[2]² - 10 x[1] + 10 x[2] + t[3] ≥ 25"
 "-x[1]² + 2 x[3]*x[1] - x[3]² - 2 x[1] + 2 x[3] + t[3] ≥ 1"
 "-x[1]² + 2 x[2]*x[1] - x[2]² - 10 x[1] + 10 x[2] + t[3] ≥ 25"
 "-x[2]² + 2 x[3]*x[2] - x[3]² + 8 x[2] - 8 x[3] + t[3] ≥ 16"
 "-x[1]² + 2 x[3]*x[1] - x[3]² - 2 x[1] + 2 x[3] + t[3] ≥ 1"
 "-x[2]² + 2 x[3]*x[2] - x[3]² + 8 

### Figure 1

In [31]:
sample = [ [0,  0, 8], 
           [0,  2, 4],
           [0,  5, 3],
           [0, 10, 2] ]

@show trop_frechet(sample) |> trop_normalize
P = trop_frechet_set(sample, tol=1e-3)

trop_frechet(sample) |> trop_normalize = [0.0, 3.205087304018976, 3.2050203078168154]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
1-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[0, 1, -1], 0//1),
2-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, -1, 0], -3//1)
 HalfSpace(Rational{BigInt}[-1, 1, 0], 4//1)

In [32]:
exact_FM = points(P) |> collect |> rand |> trop_normalize
@show exact_FM
trop_dist.(Ref(exact_FM), sample)

exact_FM = Rational{BigInt}[0, 4, 4]


4-element Vector{Rational{BigInt}}:
 8
 2
 2
 8

### Random points in high dimensional space

In [55]:
n = 20
m = n

rand_sample = [rand(1:20, n) for _ in 1:m]

20-element Vector{Vector{Int64}}:
 [10, 1, 17, 6, 10, 19, 17, 13, 11, 4, 10, 14, 17, 12, 14, 17, 18, 6, 11, 3]
 [16, 10, 14, 8, 19, 19, 6, 10, 11, 8, 17, 8, 9, 2, 5, 5, 7, 9, 13, 15]
 [9, 11, 14, 6, 5, 8, 7, 3, 9, 5, 1, 5, 11, 9, 16, 4, 6, 13, 9, 17]
 [13, 11, 14, 19, 2, 7, 7, 20, 8, 9, 11, 1, 15, 5, 11, 13, 1, 14, 19, 15]
 [8, 5, 9, 5, 9, 4, 20, 1, 8, 17, 17, 19, 18, 4, 11, 10, 16, 16, 8, 3]
 [16, 18, 3, 13, 13, 7, 14, 3, 5, 15, 16, 6, 19, 7, 1, 19, 20, 15, 2, 6]
 [1, 14, 18, 16, 17, 17, 1, 11, 6, 16, 15, 12, 8, 13, 14, 2, 9, 16, 18, 16]
 [6, 6, 7, 1, 7, 18, 6, 5, 6, 6, 19, 2, 11, 4, 6, 9, 14, 7, 7, 14]
 [4, 19, 6, 2, 7, 20, 7, 7, 17, 7, 1, 20, 17, 6, 17, 4, 16, 17, 15, 6]
 [6, 15, 9, 7, 4, 15, 20, 12, 2, 3, 2, 16, 20, 1, 17, 6, 1, 14, 5, 4]
 [9, 19, 5, 12, 17, 16, 10, 12, 16, 4, 12, 17, 18, 14, 11, 20, 4, 18, 10, 1]
 [10, 13, 10, 3, 17, 1, 11, 5, 19, 18, 12, 19, 3, 5, 18, 1, 18, 3, 7, 4]
 [15, 1, 1, 11, 8, 9, 1, 4, 13, 4, 11, 2, 2, 11, 3, 11, 9, 18, 12, 3]
 [5, 10, 18, 11, 9, 12, 3, 

In [56]:
@time trop_frechet(rand_sample) |> trop_normalize

  0.504502 seconds (3.89 M allocations: 302.356 MiB, 36.80% gc time)


20-element Vector{Float64}:
  0.0
  2.236703211533353e-8
  1.8532017365124887e-8
 -1.9999999971934712
 -0.2016807836733826
 -0.0001212024113657062
 -3.755869282251467e-9
 -1.0000000200499557
 -0.39135440956281387
 -0.9999999973742796
 -2.4666669380400514
 -0.9999999997450629
 -1.566416085729827e-8
 -2.9258028485278804
  3.3761223816597408e-9
  0.13333451922391737
 -1.0001212043827388
  1.9998805932109995
  1.0000000041693116
 -1.9999999692143269

In [57]:
@time trop_frechet_set(rand_sample, rep="vrep", tol=1e-3)

  3.094123 seconds (14.32 M allocations: 401.911 MiB, 5.50% gc time, 0.52% compilation time)


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
16-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0//1)
 HyperPlane(Rational{BigInt}[1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2//1)
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3//29410)
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 2//1)
 HyperPlane(Rational{BigInt}[0, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3//29410)
 HyperPlane(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0], 29413//29410)
 HyperPlane(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0], -58817//29410)
 HyperPlane(Rational{BigInt}[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1367//2941)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 

### Big random sample in 3-space

In [32]:
n = 3

Random.seed!(50)

for m = 2:3n
    dim_vec = zeros(Int64, n+1)
    for _ = 1:100
        rand_sample = [rand(1:10, n) for _ in 1:m]
        try
            my_frech = trop_frechet_set(rand_sample, rep="hrep", tol=1e-3)
            dim_vec[dim(my_frech) + 1] += 1
        catch e
            println(rand_sample)
        end
    end
    println(m, " sample points ", dim_vec / sum(dim_vec))
end

[[5, 5, 9], [4, 6, 9]]
2 sample points [0.0, 0.26262626262626265, 0.7373737373737373, 0.0]
3 sample points [0.0, 0.6, 0.4, 0.0]
4 sample points [0.0, 0.88, 0.12, 0.0]
5 sample points [0.0, 0.93, 0.07, 0.0]
6 sample points [0.0, 0.97, 0.03, 0.0]
[[2, 4, 10], [3, 10, 2], [3, 2, 9], [1, 3, 3], [10, 6, 4], [6, 9, 1], [10, 9, 2]]
[[2, 7, 1], [10, 3, 10], [9, 10, 10], [1, 2, 4], [5, 6, 7], [4, 6, 1], [6, 9, 10]]
[[8, 9, 1], [8, 5, 10], [10, 3, 5], [9, 10, 9], [10, 7, 6], [7, 3, 7], [1, 1, 6]]
7 sample points [0.0, 1.0, 0.0, 0.0]
8 sample points [0.0, 1.0, 0.0, 0.0]
9 sample points [0.0, 1.0, 0.0, 0.0]


## Application: Phylogenetic trees

In [27]:
using JSON3
using DataFrames

# Function to read the JSON file and convert to a list of matrices
function read_and_convert_json(file_path::String)
    # Read the JSON file
    json_data = JSON3.read(file_path)
    
    # Extract elements from the nested arrays
    elements = [x[1] for x in json_data]
    elements = rationalize.(10000 * elements, tol=1e-2)
    
    # Convert elements into matrices
    num_elements = length(elements)
    matrices = []
    
    for i in 1:64:num_elements
        # Get the next 64 elements
        matrix_elements = elements[i:min(i+63, num_elements)]
        
        # Convert to an 8x8 matrix if there are 64 elements, otherwise create a smaller matrix
        matrix_size = length(matrix_elements)
        sqrt_size = Int(sqrt(matrix_size))
        push!(matrices, reshape(matrix_elements, sqrt_size, sqrt_size))
    end
    
    return matrices
end

# Read and convert the JSON file
file_path = "all_matrices.json"
matrices = read_and_convert_json(file_path)
taxa = ["Tg", "Et", "Cp", "Ta", "Bb", "Tt", "Pv", "Pf"]

8-element Vector{String}:
 "Tg"
 "Et"
 "Cp"
 "Ta"
 "Bb"
 "Tt"
 "Pv"
 "Pf"

In [28]:
"""
Take a matrix of pairwise distances between taxa and returns the cophenetic vector.
"""
function cophenetic_from_distance(pairwise)
    n = size(pairwise, 1)
    coph = [pairwise[i, j] for i in 1:n-1 for j in i+1:n]
    return coph
end

cophenetic_from_distance

In [29]:
"""
Check if a distance matrix defines a phylogenetic tree
"""
function is_phylogenetic_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the four-point condition
    for i in 1:n-3
        for j in i+1:n-2
            for k in j+1:n-1
                for l in k+1:n
                    # Calculate distances
                    D_ij_kl = D[i, j] + D[k, l]
                    D_ik_jl = D[i, k] + D[j, l]
                    D_il_jk = D[i, l] + D[j, k]
                    
                    # Check the four-point condition
                    if !(D_ij_kl >= D_ik_jl && D_ij_kl >= D_il_jk) &&
                       !(D_ik_jl >= D_ij_kl && D_ik_jl >= D_il_jk) &&
                       !(D_il_jk >= D_ij_kl && D_il_jk >= D_ik_jl)
                        return false
                    end
                end
            end
        end
    end
    
    return true
end

is_phylogenetic_tree

In [30]:
"""
Check if a distance matrix defines an ultrametric tree
"""
function is_ultrametric_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the ultrametric condition
    for i in 1:n-2
        for j in i+1:n-1
            for k in j+1:n
                # Calculate distances
                Dij = D[i, j]
                Dik = D[i, k]
                Djk = D[j, k]
                
                # Check if the largest distance is attained at least twice
                if !(Dij <= max(Dik, Djk) && Dik <= max(Dij, Djk) && Djk <= max(Dij, Dik))
                    return false
                end
            end
        end
    end
    
    return true

end

is_ultrametric_tree

In [31]:
println("All phylogenetic: ", all([is_phylogenetic_tree(matrices[i]) for i = 1:268]))
println("All ultrametric: ", all([is_ultrametric_tree(matrices[i]) for i = 1:268]))

All phylogenetic: true
All ultrametric: false


In [32]:
coph_vecs = [cophenetic_from_distance(mat) for mat in matrices]
phylo_frech = trop_frechet_set(coph_vecs, rep="hrep")

Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
4-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 0//1)
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 0//1)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1),
294-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 866//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 

In [59]:
length(coph_vecs)

268

In [39]:
dim(phylo_frech)

24

In [40]:
N = 10
M = 100
Random.seed!(123)

dimensions = [[] for _ = 3:N]
for n = 3:N
    for m = 2:M
        sample = [Rational.(rand(0:10000, n)) for _ in 1:m]
        push!(dimensions[n-2], dim(trop_frechet_set(sample, rep="hrep")))
        println("m = ", m, "; n = ", n, " done!")
    end
end

dimensions

m = 2; n = 3 done!
m = 3; n = 3 done!
m = 4; n = 3 done!
m = 5; n = 3 done!
m = 6; n = 3 done!
m = 7; n = 3 done!
m = 8; n = 3 done!
m = 9; n = 3 done!
m = 10; n = 3 done!
m = 11; n = 3 done!
m = 12; n = 3 done!
m = 13; n = 3 done!
m = 14; n = 3 done!
m = 15; n = 3 done!
m = 16; n = 3 done!
m = 17; n = 3 done!
m = 18; n = 3 done!
m = 19; n = 3 done!
m = 20; n = 3 done!
m = 21; n = 3 done!
m = 22; n = 3 done!
m = 23; n = 3 done!
m = 24; n = 3 done!
m = 25; n = 3 done!
m = 26; n = 3 done!
m = 27; n = 3 done!
m = 28; n = 3 done!
m = 29; n = 3 done!
m = 30; n = 3 done!
m = 31; n = 3 done!
m = 32; n = 3 done!
m = 33; n = 3 done!
m = 34; n = 3 done!
m = 35; n = 3 done!
m = 36; n = 3 done!
m = 37; n = 3 done!
m = 38; n = 3 done!
m = 39; n = 3 done!
m = 40; n = 3 done!
m = 41; n = 3 done!
m = 42; n = 3 done!
m = 43; n = 3 done!
m = 44; n = 3 done!
m = 45; n = 3 done!
m = 46; n = 3 done!
m = 47; n = 3 done!
m = 48; n = 3 done!
m = 49; n = 3 done!
m = 50; n = 3 done!
m = 51; n = 3 done!
m = 52; 

8-element Vector{Vector{Any}}:
 [2, 1, 2, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [3, 2, 1, 2, 1, 2, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [4, 3, 3, 3, 2, 1, 2, 1, 2, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [5, 5, 4, 4, 3, 4, 1, 2, 2, 3  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [7, 7, 6, 3, 5, 4, 2, 4, 4, 4  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [7, 8, 5, 4, 6, 4, 5, 6, 2, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [8, 7, 8, 7, 6, 5, 6, 5, 3, 4  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [9, 9, 10, 6, 7, 6, 4, 5, 5, 6  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [41]:
using Plots

# Example data: categories and their respective proportions in different groups
categories = ["Category 1", "Category 2", "Category 3"]

proportional_dimensions = [dim_list / M for dim_list in dimensions]
data = hcat(proportional_dimensions...)

# Create the proportional bar chart
bar(data, label=1:M, legend=:topright, title="Proportional Bar Chart", xlabel="Groups", ylabel="Proportion", bar_width=0.7, lw=0, series_annotations=transpose([group1 group2 group3]))

# Save the plot to a file
savefig("proportional_bar_chart.png")

# Display the plot
display(plot)

UndefVarError: UndefVarError: `group1` not defined

In [42]:
for thing in dimensions
    println(thing)
    println()
end

Any[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Any[3, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Any[4, 3, 3, 3, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Any[5, 5, 4, 4, 3, 4, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Divide-and-conquer

In [32]:
coph_vecs = [cophenetic_from_distance(mat) for mat in matrices]
@time trop_frechet_set(coph_vecs)

124.374851 seconds (389.82 M allocations: 8.778 GiB, 1.51% gc time, 0.22% compilation time)


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
4-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 0//1)
 HyperPlane(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 0//1)
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1),
294-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 866//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 

In [33]:
chunks = map(1:26) do i
    coph_vecs[(i-1)*10+1:i*10]
end

26-element Vector{Vector{Vector{Rational{Int64}}}}:
 [[3784, 6626, 9906, 6521, 11778, 8750, 7661, 7601, 10881, 7496  …  4901, 15579, 12551, 11462, 12194, 9167, 8078, 14217, 13128, 1089], [3485, 8484, 9427, 8865, 25257, 9257, 10300, 8814, 9756, 9195  …  3645, 25010, 9010, 10054, 24449, 8449, 9492, 20033, 21076, 2163], [3570, 5592, 6890, 4591, 4849, 5136, 4872, 3905, 6062, 3763  …  6609, 6867, 7154, 6890, 2383, 4367, 4103, 4625, 4361, 274], [1696, 4297, 5665, 6408, 4431, 4218, 4248, 4473, 5841, 6584  …  9039, 7062, 6849, 6880, 5542, 6984, 7014, 5006, 5037, 557], [1864, 8173, 6779, 10056, 10843, 9010, 9319, 8433, 7039, 10315  …  11759, 12546, 10713, 11022, 15116, 13283, 13592, 12332, 12642, 2722], [3801, 11567, 5222, 5725, 5642, 8167, 7548, 12836, 6491, 6994  …  6129, 6046, 8571, 7952, 601, 7410, 6791, 7327, 6708, 4443], [751, 29706, 6877, 4376, 5857, 9999, 8966, 29899, 7070, 4569  …  6079, 7560, 11702, 10669, 3359, 8305, 7272, 9787, 8754, 5104], [4080, 8200, 9699, 9924, 11942, 9302, 9463

In [34]:
@time trop_frechet_set(chunks[1])

 21.307364 seconds (60.00 M allocations: 835.816 MiB, 0.19% gc time, 0.08% compilation time)


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
1-element iterator of HyperPlane{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HyperPlane(Rational{BigInt}[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 9636073//1218),
80-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -12812617//1218)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 10541281//2562)
 HalfSpace(Rational{BigInt}[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], 6969853//2562)
 HalfSpace(Rational{BigInt}[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -8397367//1218)
 HalfSpace(Rational{BigInt}[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -6593509//1218)
 HalfSpace(Rati

In [ ]:
using ProgressBars

runtimes = []
intermediate_FM = []
@time begin 
    for P in ProgressBar(chunks)
        @time push!(runtimes, @elapsed push!(intermediate_FM, trop_frechet_set(P) .|> points .|> first))
    end
    trop_frechet_set(intermediate_FM)
end